# Find lidar ratio

This code helps you to find the lidar ratio by means of the comparison of the lidar and sun-photometer AOD. If you are not confident in Python, don't worry! You only need to make modifications to the statements found as follows:

#------------------------------------------------------------------------------------------------------------------------------

         Modify ONLY within here
#------------------------------------------------------------------------------------------------------------------------------

Important:
- A recurring error may be that paths are specified with '\\' as the folder separator, while the Python language uses '\\\\' (double).
- It is also recommended to run the cells one by one, and not to re-run a cell while it is executing (the software may hang). If while a cell is running, any changes are made to the code in that cell and we want to re-run it, we should wait for it to finish or restart the kernels.
- If desired, you can play with the rest of the code.

## IMPORTING LIBRARIES AND SPECIFYING PATHS

First, we need to import the necessary libraries. In case any of the modules are not found when running, install the module using:

#------------------------------------------------------------------------------------------------------------------------------

    !pip install <module_name>
#------------------------------------------------------------------------------------------------------------------------------


In [1]:
from pathlib import Path
import datetime as dt
import numpy as np #For general mathematical calculations
import matplotlib.pyplot as plt #For plotting

#For working with lidar products (developed by the Atmospheric Physics Group of the UGR, GFAT)
from gfatpy.lidar.preprocessing import preprocess
from gfatpy.lidar.plot.quicklook import quicklook_xarray
from gfatpy.utils.plot import font_axes
from gfatpy.atmo.ecmwf import get_ecmwf_temperature_pressure
from gfatpy.atmo.freudenthaler_molecular_properties import molecular_properties
from gfatpy.lidar.utils import refill_overlap, signal_to_rcs
from gfatpy.lidar.retrieval.klett import find_lidar_ratio
from gfatpy.aeronet.reader import reader, header
from gfatpy.aeronet.utils import find_aod
from gfatpy.utils.utils import numpy_to_datetime
%load_ext autoreload
%autoreload 2

%matplotlib inline

In the next cell, we define the directory where the lidar measurement files are located and the directory where we want to save the graphs.

In [2]:
#------------------------------------------------------------------------------------------------------------------------------
lidar_dir = Path(r'C:\Users\Usuario\Mi unidad\04. Proyectos\ATMO-ACCESS\curso\data\lidar')  #Database folder path, until '\1a' (included). Example: 'C:\\Users\\user\\Desktop\\data\\1a'
lidar_dir = Path(r'C:\Users\Usuario\Documents\gitlab\gfatpy\tests\datos\ALHAMBRA\1a\2023\02\22')
aeronet_dir = Path(r'C:\Users\Usuario\Mi unidad\04. Proyectos\ATMO-ACCESS\curso\data\aeronet')  #Database folder path, until '\1a' (included). Example: 'C:\\Users\\user\\Desktop\\data\\1a'
output_dir = Path(r'C:\Users\Usuario\Mi unidad\04. Proyectos\ATMO-ACCESS\curso\output')  #Output folder path. Example: ''C:\\Users\\user\\Desktop\\output''
#------------------------------------------------------------------------------------------------------------------------------

lidar_dir.is_dir(), output_dir.is_dir() #Comprueba que ambas carpetas existen.


(False, False)

## LOAD AERONET DATA

AERONET data are read with `reader`. This program can read AERONET files with extentions `*.all` and `*.lev15`.

In [ ]:
aeronet_wildcard = (aeronet_dir /  '*Granada.all').absolute().as_posix()
aeronet_header, aeronet = reader(aeronet_wildcard)

In [ ]:
aeronet_header

## Read lidar data


In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
RS_FL = lidar_dir / 'alh_1a_Prs_rs_xf_20230222.nc' #lidar file
range_tuple = (0, 10000.0) #Range range to be used
#------------------------------------------------------------------------------------------------------------------------------

#Load and preprocess and range selection 
whole_lidar = preprocess(RS_FL, crop_ranges=range_tuple)

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
time_tuple = ('2023-02-22 11:00:00', '2023-02-22 13:40:00') #Time range to be used
#------------------------------------------------------------------------------------------------------------------------------

#time selection from lidar data
lidar = whole_lidar.sel(time=slice(*time_tuple))

#Dataset rolling (from now on each profile is 30min average)
lidar = lidar.rolling(time=30).mean()

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
channels = ['532fta']
allowed_time_gap_hour = 3 #Time gap between the time when the AERONET AOD was measured and the lidar profile
reference_range = (7000, 8000)
#-----------------------------------------------------------------------------------------------------------------------------

range_array = lidar.range.values

lr = {}

for channel_ in channels:
    lr[channel_] = np.nan*np.ones(len(lidar.time))
    # Wavelength
    wavelength = int(channel_[:-3])
    
    lr[wavelength] = []
    for idx, time_ in enumerate(lidar.time):                
        
        #Profile
        rcs = signal_to_rcs(lidar[f'signal_{channel_}'].sel(time=time_), lidar.range).values

        if not np.isnan(rcs).all():

            #Convert time
            dt_time_ = numpy_to_datetime(time_.values)

            measured_aod, measured_datetime = find_aod(aeronet_header, aeronet, target_wavelength = wavelength, target_datetime=dt_time_, allowed_time_gap_hour=allowed_time_gap_hour)

            if measured_aod != None:

                #ECMWF      
                date_str =  dt_time_.strftime("%Y%m%d") #lidar file
                hour = int(dt_time_.strftime('%H'))

                #Read ECMWF data
                atmo = get_ecmwf_temperature_pressure(date_str, hour=hour, heights=range_array)

                #Retrieve molecular properties at elastic wavelengths     
                from pdb import set_trace

                mol_properties = molecular_properties(
                        wavelength, atmo["pressure"].values, atmo["temperature"].values, heights=lidar.range
                    )

                #Beta mol
                beta_mol = mol_properties['molecular_beta'].values
                lr_mol = mol_properties['molecular_lidar_ratio'].item()

                #Find lidar ratio
                lr_, rel_diff_aod, success = find_lidar_ratio(rcs, 
                                                                              range_array, 
                                                                              beta_mol, 
                                                                              lr_mol,
                                                                              measured_aod, 
                                                                              **{'reference':reference_range}, 
                                                                              debugging=False)
                lr[wavelength].append(lr_)
                print([wavelength, lr_, rel_diff_aod, success])
            else: 
                print('Not AOD found.')
    
LR = xr.Dataset()
for channel_ in channels:
    LR['channel_'] =  xr.DataArray(lr[channel_], dims=['time'], coords={'time': profile_time})

lr_filepath = output_dir / f'lr_{datestr}.nc'
LR.to_netcdf(lr_filepath)